In [ ]:
from pathlib import Path
import pandas as pd
import seaborn as sns
from optuna import Study
from src.utils import Run, Sweep, set_directory, INFERENCE_LABELS, EXPERIMENT_PATH
from hydra.utils import instantiate
import torch
from tqdm import tqdm
from src.models.base import ErrorRate

In [ ]:
dir_ = (EXPERIMENT_PATH / "cifar10_small" / "2021-12-22" / "13-59-55")
runs =  [Run(p) for p in dir_.glob("[012]")]

In [ ]:
dm = instantiate(runs[0].cfg.data)
dm.setup("test")

inference = instantiate(runs[0].cfg.inference)
ckpt = next(iter((runs[0].dir / "checkpoints").glob("epoch*.ckpt")))
inference.load_state_dict(torch.load(ckpt, map_location="cpu")["state_dict"])

In [ ]:
inference.model

In [ ]:
def get_dropout_ensemble(inference, input, n=10):

    inference.model.train()
    output = 0
    with torch.no_grad():
        for _ in range(n):
            output += inference.model(input).softmax(-1)
        output /= n
    return output
        
err = ErrorRate()

for x, y in tqdm(dm.test_dataloader()):
    pred = get_dropout_ensemble(inference, x)
    err.update(pred, y)

    pass

err.compute()

In [ ]:
err.update()